In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import re
import enum
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp '/content/drive/MyDrive/FPL-Project/FPL-Utils.ipynb' /content/
%run /content/FPL-Utils.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir()

['Enum',
 'HTTPError',
 'In',
 'Out',
 '_',
 '_3',
 '_5',
 '_6',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'calculate_point_ratio_against_team',
 'calculate_team_cumulative_goals_conceded',
 'calculate_team_cumulative_goals_scored',
 'chardet',
 'clean_name',
 'conceded_df_combined',
 'create_conceded_form_features',
 'create_goalkeepers_form_features',
 'create_goals_assissts_points_form_features',
 'create_scored_form_features',
 'detect_encoding',
 'df',
 'drive',
 'enum',
 'exit',
 'get_current_season_data',
 'get_ipython',
 'get_teams_previous_position',
 'get_teams_previous_position_2020_21',
 'get_teams_previous_position_2021_2022',
 'get_teams_previous_position_2022_23',
 'get_teams_previous_position_2023_24',
 'get_was_home_values',
 'gk_extract',
 'js

In [ ]:

df = pd.read_csv(
        "/content/drive/MyDrive/FPL-Project/cleaned_previous_seasons_gameweeks.csv",
        encoding='utf-8',
    )

print("Calculating Goal Assists Points Form")
df = create_goals_assissts_points_form_features(df)
print("Calculating Goalkeepers Form")
gk_extract = create_goalkeepers_form_features(df)
print("Calculating Conceded Form")
conceded_df_combined = create_conceded_form_features(df)
print("Calculating Scored Form")
scored_df_combined = create_scored_form_features(df)

df['team_cumulative_goals_scored'] = 0
df['team_cumulative_goals_scored_5_match_form'] = 0
df['team_cumulative_goals_conceded'] = 0
df['team_cumulative_goals_conceded_5_match_form'] = 0
df['gk_form_goals_conceded_5_match_form'] = 0

df.update(scored_df_combined[['season', 'team', 'GW', 'team_cumulative_goals_scored', 'team_cumulative_goals_scored_5_match_form']])
df.update(conceded_df_combined[['season', 'team', 'GW', 'team_cumulative_goals_conceded', 'team_cumulative_goals_conceded_5_match_form']])
df.update(gk_extract[['season', 'name', 'GW', 'gk_form_goals_conceded_5_match_form']])

df.to_csv('/content/drive/MyDrive/FPL-Project/updated_cleaned_previous_seasons_gameweeks_with_features.csv', index=False)

<ipython-input-12-8f677526372a>:1: DtypeWarning: Columns (58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Calculating for 2020-21
Calculating for 2021-22
Calculating for 2022-23


<ipython-input-10-bcf7a699bef5>:301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceded_df_combined['team_cumulative_goals_conceded_5_match_form'] = conceded_df.groupby(['season', 'team'])['team_cumulative_goals_conceded'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())


Calculating for 2020-21
Calculating for 2021-22
Calculating for 2022-23


<ipython-input-10-bcf7a699bef5>:333: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scored_df_combined['team_cumulative_goals_scored_5_match_form'] = scored_df.groupby(['season', 'team'])['team_cumulative_goals_scored'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())


In [ ]:
df[(df['team'] == 'Brighton') & (df['season'] == '2020-21')]

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,expected_goals_conceded,starts,total_points_5_match_form,assists_5_match_form,goals_5_match_form,team_cumulative_goals_scored,team_cumulative_goals_scored_5_match_form,team_cumulative_goals_conceded,team_cumulative_goals_conceded_5_match_form,gk_form_goals_conceded_5_match_form
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,NaN,NaN,1.000000,0.0,0.000000,1,1.0,3,3.0,0.0
425,Aaron Connolly,FWD,Brighton,4.0,0,2,27,1,11.3,78,...,NaN,NaN,4.500000,0.0,0.500000,4,4.0,3,3.0,0.0
964,Aaron Connolly,FWD,Brighton,2.7,0,0,2,0,12.1,78,...,NaN,NaN,3.666667,0.0,0.333333,6,6.0,6,6.0,0.0
1518,Aaron Connolly,FWD,Brighton,2.7,0,0,7,0,0.3,78,...,NaN,NaN,3.250000,0.0,0.250000,8,8.0,10,10.0,0.0
2080,Aaron Connolly,FWD,Brighton,3.0,1,0,13,0,10.3,78,...,NaN,NaN,3.400000,0.2,0.200000,9,9.0,11,11.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21170,Yves Bissouma,MID,Brighton,2.8,0,0,10,1,0.6,76,...,NaN,NaN,2.600000,0.0,0.000000,35,35.0,39,39.0,0.0
22111,Yves Bissouma,MID,Brighton,0.0,0,0,10,0,0.9,76,...,NaN,NaN,2.400000,0.0,0.000000,36,36.0,41,41.0,0.0
22811,Yves Bissouma,MID,Brighton,1.5,0,0,12,0,14.1,76,...,NaN,NaN,2.000000,0.0,0.000000,37,37.0,42,42.0,0.0
23480,Yves Bissouma,MID,Brighton,1.0,0,0,18,0,14.0,76,...,NaN,NaN,1.800000,0.0,0.000000,40,40.0,44,44.0,0.0
